In [ ]:
!pip install youtube-transcript-api

In [ ]:
import requests
import pandas as pd
import deepl
from datetime import datetime, timedelta
import numpy as np
from google.colab import files
from tqdm.notebook import tqdm
import time

tqdm.pandas()

In [ ]:
from googleapiclient.discovery import build

from youtube_transcript_api import YouTubeTranscriptApi

In [ ]:
# API Key was removed

In [ ]:
youtube = build("youtube", "v3", developerKey=api_key)

# Channel ID finder

In [ ]:
CHANNEL_NAMES = ["@DiarioElComercio", "@atvnoticias"]


# Defining function that fetches channel IDs
def get_channel_ids(api_key, channel_names):

    base_url = "https://www.googleapis.com/youtube/v3/search"
    results = {}

    for name in channel_names:
        params = {
            "part": "snippet",
            "q": name,
            "type": "channel",
            "key": api_key
        }

        response = requests.get(base_url, params=params).json()

        if "items" in response and response["items"]:
            channel_id = response["items"][0]["id"]["channelId"]
            results[name] = channel_id
        else:
            results[name] = "Not Found"

    return results



# Fetch Channel IDs
channel_ids = get_channel_ids(api_key, CHANNEL_NAMES)

for name, channel_id in channel_ids.items():
    print(f"{name} - Channel ID: {channel_id}")


In [ ]:
# Channel id ATV Noticias: UCYG5uXS3xdsoaXIxum1pAEw
# Channel id El Comercio: UCLtGUPjKLqa3zgdmhKCZONg

# Collecting videos IDs for ATV Noticias





## Full month periods

In [ ]:
#Dataframe with full month periods

# Define the start and end dates
start_date = datetime(2019, 2, 1)
end_date = datetime(2025, 3, 1)

data = []

# Generate the start and end date-times for each full month period
current_date = start_date
while current_date < end_date:
    # Get the start date of the current month
    start_date_str = current_date.strftime("%Y-%m-%dT%H:%M:%S-05:00")

    # Calculate the next month's first day for the end date
    next_month = current_date.replace(day=28) + timedelta(days=4)  # this gives us the next month
    end_date_str = next_month.replace(day=1) - timedelta(seconds=1)
    end_date_str = end_date_str.strftime("%Y-%m-%dT%H:%M:%S-05:00")

    # Append to the data list
    data.append([start_date_str, end_date_str])

    # Move to the first day of the next month
    current_date = next_month.replace(day=1)

date_ranges = pd.DataFrame(data, columns=["start", "end"])

date_ranges


In [ ]:
# Initial test
channel_id = "UCYG5uXS3xdsoaXIxum1pAEw" # ATV

# Defined keywords
keyword = "robo|robó|roban|robaron|delincuentes|criminales|asalto|asaltó|asaltan|asaltaron|matan|mató|mataron|ladrón|asesinato|asesinado|asesinan|asesinaron|extorsionadores|extorsionador|extorsionan|extorsionaron|extorsionó|secuestro|secuestradores|secuestrada|secuestrado|secuestró|secuestraron|balacera|balean|balearon|balazos|sicariato|mafias|mafia|sicarios"

In [ ]:
# Function used for the first sample
#Function for getting video info for one channel id, one keyword, one time frame
# Returns dictionary of all videos found, and prints number

def get_youtube_videos(api_key, channel_id, keyword, max_results=50, order="date",
                       published_after=None, published_before=None):

    videos = []
    # Prepare API request parameters
    params = {
        "part": "snippet",
        "channelId": channel_id,
        "q": keyword,
        "type": "video",
        "maxResults": max_results,
        "order": order,
    }

    if published_after:
        params["publishedAfter"] = published_after

    if published_before:
        params["publishedBefore"] = published_before

    # Execute API request
    request = youtube.search().list(**params)
    response = request.execute()

    # Extract video details
    for item in response.get("items", []):
        video_id = item["id"]["videoId"]
        video_title = item["snippet"]["title"]
        video_url = f"https://www.youtube.com/watch?v={video_id}"
        video_date = item["snippet"]["publishedAt"]

        videos.append({
            "video_id": video_id,
            "title": video_title,
            "url": video_url,
            "channel": channel_id,
            "keyword": keyword,
            "video_date": video_date})

    print("Number of videos found between " + published_after + " and " + published_before + ": " + str(len(videos)))

    return videos

In [ ]:
# First saample creation
# Create a for loop that goes through each time frame
    # gets all youtube videos and store them in a dataframe

channel_id = "UCYG5uXS3xdsoaXIxum1pAEw" # ATV
max_results = 50
order = "date"
keyword = "robo|robó|roban|robaron|delincuentes|criminales|asalto|asaltó|asaltan|asaltaron|matan|mató|mataron|ladrón|asesinato|asesinado|asesinan|asesinaron|extorsionadores|extorsionador|extorsionan|extorsionaron|extorsionó|secuestro|secuestradores|secuestrada|secuestrado|secuestró|secuestraron|balacera|balean|balearon|balazos|sicariato|mafias|mafia|sicarios"

# Create empty dataframe for first time running it
df = pd.DataFrame(columns=['video_id','title','url','channel','keyword','video_date'])

for index, row in date_ranges.iterrows():
    published_after = row['start']
    published_before = row['end']

    df_new = pd.DataFrame.from_records(get_youtube_videos(api_key, channel_id, keyword, max_results,
                                                          order, published_after, published_before))
    df = pd.concat([df, df_new], ignore_index=True)

In [ ]:
df.shape

In [ ]:
# Save df as csv
df.to_csv("video_titles.csv")

In [ ]:

# new function
#Function for getting video info for one channel id, one keyword, one time frame
# Returns dictionary of all videos found and number of videos, and prints number
#

def get_youtube_videos(api_key, channel_id, keyword, max_results=50, order="date",
                       published_after=None, published_before=None):

    videos = []

    # Prepare API request parameters
    params = {
        "part": "snippet",
        "channelId": channel_id,
        "q": keyword,
        "type": "video",
        "maxResults": max_results,
        "order": order,
    }

    if published_after:
        params["publishedAfter"] = published_after

    if published_before:
        params["publishedBefore"] = published_before

    # Execute API request
    request = youtube.search().list(**params)
    response = request.execute()

    # Extract video details
    for item in response.get("items", []):
        video_id = item["id"]["videoId"]
        video_title = item["snippet"]["title"]
        video_url = f"https://www.youtube.com/watch?v={video_id}"
        video_date = item["snippet"]["publishedAt"]

        videos.append({
            "video_id": video_id,
            "title": video_title,
            "url": video_url,
            "channel": channel_id,
            "keyword": keyword,
            "video_date": video_date})


    print("Number of videos found between " + published_after + " and " + published_before + ": " + str(len(videos)))



    return videos, len(videos)

In [ ]:

# applying new function
# Create a for loop that goes through each time frame
    # gets all youtube videos and store them in a dataframe

channel_id = "UCYG5uXS3xdsoaXIxum1pAEw" # ATV
max_results = 50
order = "date"
keyword = "robo|robó|roban|robaron|delincuentes|criminales|asalto|asaltó|asaltan|asaltaron|matan|mató|mataron|ladrón|asesinato|asesinado|asesinan|asesinaron|extorsionadores|extorsionador|extorsionan|extorsionaron|extorsionó|secuestro|secuestradores|secuestrada|secuestrado|secuestró|secuestraron|balacera|balean|balearon|balazos|sicariato|mafias|mafia|sicarios"

# Create empty dataframe for first time running it
df = pd.DataFrame(columns=['video_id','title','url','channel','keyword','video_date'])
videos_list = []
videos_count_list = []

for index, row in date_ranges.iterrows():
    published_after = row['start']
    published_before = row['end']


    all = get_youtube_videos(api_key, channel_id, keyword, max_results,
                                                          order, published_after, published_before)
    videos = all[0]
    video_count = all[1]
    videos_list.append(videos)
    videos_count_list.append(video_count)


In [ ]:
flattened_list = np.concatenate(videos_list).tolist()
df1 = pd.DataFrame(flattened_list)
df1.drop(columns=['keyword'], inplace=True)

In [ ]:
df1.head()

In [ ]:
df1.to_csv("videos_ATV_1.csv")

In [ ]:
date_ranges['videos_count'] = videos_count_list

In [ ]:
date_ranges.to_csv("date_ranges_ATV_1.csv")

In [ ]:
date_ranges.tail()

## Half-month periods

In [ ]:
#creating a new df where the time ranges of those that have vidoe count == 50 are split in half

# Convert start and end columns to datetime
date_ranges["start"] = pd.to_datetime(date_ranges["start"])
date_ranges["end"] = pd.to_datetime(date_ranges["end"])

df_filtered = date_ranges[date_ranges["videos_count"] == 50].copy()

# Create new df
split_periods = []

for _, row in df_filtered.iterrows():
    start_date = row["start"]
    mid_date = start_date + timedelta(days=15)
    end_date = row["end"]

    # Ensure mid_date does not exceed end_date
    if mid_date > end_date:
        mid_date = end_date

    # First half
    split_periods.append({"start": start_date, "end": mid_date - timedelta(seconds=1)})

    # Second half
    split_periods.append({"start": mid_date, "end": end_date})

# Create new DataFrame with split periods
df_half_months = pd.DataFrame(split_periods)

# required format
df_half_months["start"] = df_half_months["start"].dt.strftime("%Y-%m-%dT%H:%M:%S-05:00")
df_half_months["end"] = df_half_months["end"].dt.strftime("%Y-%m-%dT%H:%M:%S-05:00")

print(df_half_months)

In [ ]:
# Getting the youtube videos for the split periods

channel_id = "UCYG5uXS3xdsoaXIxum1pAEw" # ATV
max_results = 50
order = "date"
keyword = "robo|robó|roban|robaron|delincuentes|criminales|asalto|asaltó|asaltan|asaltaron|matan|mató|mataron|ladrón|asesinato|asesinado|asesinan|asesinaron|extorsionadores|extorsionador|extorsionan|extorsionaron|extorsionó|secuestro|secuestradores|secuestrada|secuestrado|secuestró|secuestraron|balacera|balean|balearon|balazos|sicariato|mafias|mafia|sicarios"

# Create empty dataframe for first time running it
df = pd.DataFrame(columns=['video_id','title','url','channel','keyword','video_date'])
videos_list = []
videos_count_list = []

for index, row in df_half_months.iterrows():
    published_after = row['start']
    published_before = row['end']


    all = get_youtube_videos(api_key, channel_id, keyword, max_results,
                                                          order, published_after, published_before)
    videos = all[0]
    video_count = all[1]
    videos_list.append(videos)
    videos_count_list.append(video_count)

In [ ]:
videos_list

In [ ]:
#creating a df with the outputs
flattened_list = np.concatenate(videos_list).tolist()
df3 = pd.DataFrame(flattened_list)
df3.drop(columns=['keyword'], inplace=True)

In [ ]:
df3.tail()

In [ ]:
# Creating new df_half_months only from feb 2024 and on

subset_df_half_months = df_half_months.iloc[26:]

subset_df_half_months

In [ ]:
# Merge monthly and partial half monthly df for ATV: df1 and df3

In [ ]:
df1.shape

In [ ]:
df3.shape

In [ ]:
ATV_df_1 = pd.concat([df1, df3], ignore_index=True).drop_duplicates()

ATV_df_1.shape

In [ ]:
# Save to continue tomorrow with more request

ATV_df_1.to_csv("ATV_df_1.csv")

In [ ]:
# Save number of videos per month period and half month period, to know which ones to iterate again

# Months
# monthly_videos = pd.read_csv("date_ranges_ATV_1.csv")

# Half months
df_half_months['videos_count'] = videos_count_list + [0] * (len(df_half_months) - len(videos_count_list))
df_half_months.to_csv("date_ranges_ATV_2.csv")

In [ ]:
df_half_months.shape

In [ ]:
# Get previous date ranges

# Months
monthly_videos = pd.read_csv("date_ranges_ATV_1.csv")

# Half months
df_half_months = pd.read_csv("date_ranges_ATV_2.csv")
df_half_months.head()

In [ ]:
subset_df_half_months = df_half_months[df_half_months["videos_count"] == -0].copy()
subset_df_half_months.head()

In [ ]:
# Running the function again from Jan 2024

# Getting the youtube videos for the split periods

channel_id = "UCYG5uXS3xdsoaXIxum1pAEw" # ATV
max_results = 50
order = "date"
keyword = "robo|robó|roban|robaron|delincuentes|criminales|asalto|asaltó|asaltan|asaltaron|matan|mató|mataron|ladrón|asesinato|asesinado|asesinan|asesinaron|extorsionadores|extorsionador|extorsionan|extorsionaron|extorsionó|secuestro|secuestradores|secuestrada|secuestrado|secuestró|secuestraron|balacera|balean|balearon|balazos|sicariato|mafias|mafia|sicarios"

# Create empty dataframe for first time running it
#df = pd.DataFrame(columns=['video_id','title','url','channel','keyword','video_date'])
videos_list = []
videos_count_list = []

for index, row in subset_df_half_months.iterrows():
    published_after = row['start']
    published_before = row['end']


    all = get_youtube_videos(api_key, channel_id, keyword, max_results,
                                                          order, published_after, published_before)
    videos = all[0]
    video_count = all[1]
    videos_list.append(videos)
    videos_count_list.append(video_count)

In [ ]:
print(len(videos_count_list))
print(subset_df_half_months.shape)

In [ ]:
#adding the video counts to the date ranges table
subset_df_half_months['videos_count'] = videos_count_list

In [ ]:
# Create dataframe with newly collected videos

flattened_list = np.concatenate(videos_list).tolist()
ATV_halfmonths_2 = pd.DataFrame(flattened_list)
ATV_halfmonths_2.drop(columns=['keyword'], inplace=True)
ATV_halfmonths_2.head()

In [ ]:
ATV_halfmonths_2.shape

In [ ]:
ATV_halfmonths_2.to_csv("ATV_halfmonths_2.csv")

In [ ]:
subset_df_half_months.to_csv("date_ranges_ATV_3.csv")

In [ ]:
#Create df that combines initial half months list, with new subset

combined_half_months = pd.concat([df_half_months.iloc[:25], subset_df_half_months], ignore_index=True)
combined_half_months.head()

In [ ]:
print(combined_half_months.shape)
print(df_half_months.shape)

In [ ]:
count50 = combined_half_months[combined_half_months["videos_count"] == 50]
count50.shape

In [ ]:
count50.tail()

## Week periods

In [ ]:
#splitting to weekly periods
split_periods = []

df_filtered = combined_half_months[combined_half_months["videos_count"] == 50].copy()

# Convert start and end columns to datetime
df_filtered["start"] = pd.to_datetime(df_filtered["start"])
df_filtered["end"] = pd.to_datetime(df_filtered["end"])

# Create new DataFrame for half-month periods
split_periods = []

for _, row in df_filtered.iterrows():
    start_date = row["start"]
    mid_date = start_date + timedelta(days=7)
    end_date = row["end"]

    # Ensure mid_date does not exceed end_date
    if mid_date > end_date:
        mid_date = end_date

    # First half
    split_periods.append({"start": start_date, "end": mid_date - timedelta(seconds=1)})

    # Second half
    split_periods.append({"start": mid_date, "end": end_date})

# Create new DataFrame with split periods
df_weeks = pd.DataFrame(split_periods)

# Convert back to the required format
df_weeks["start"] = df_weeks["start"].dt.strftime("%Y-%m-%dT%H:%M:%S-05:00")
df_weeks["end"] = df_weeks["end"].dt.strftime("%Y-%m-%dT%H:%M:%S-05:00")

print(df_weeks)

In [ ]:
df_weeks.head()

In [ ]:
# Getting the youtube videos for the weekly periods
channel_id = "UCYG5uXS3xdsoaXIxum1pAEw" # ATV
max_results = 50
order = "date"
keyword = "robo|robó|roban|robaron|delincuentes|criminales|asalto|asaltó|asaltan|asaltaron|matan|mató|mataron|ladrón|asesinato|asesinado|asesinan|asesinaron|extorsionadores|extorsionador|extorsionan|extorsionaron|extorsionó|secuestro|secuestradores|secuestrada|secuestrado|secuestró|secuestraron|balacera|balean|balearon|balazos|sicariato|mafias|mafia|sicarios"

# Create empty dataframe for first time running it
#df = pd.DataFrame(columns=['video_id','title','url','channel','keyword','video_date'])
videos_list = []
videos_count_list = []

for index, row in df_weeks.iterrows():
    published_after = row['start']
    published_before = row['end']


    all = get_youtube_videos(api_key, channel_id, keyword, max_results, order, published_after, published_before)
    videos = all[0]
    video_count = all[1]
    videos_list.append(videos)
    videos_count_list.append(video_count)


In [ ]:
#creating a df with the outputs
flattened_list = np.concatenate(videos_list).tolist()
df4 = pd.DataFrame(flattened_list)
df4.drop(columns=['keyword'], inplace=True)

In [ ]:
#adding the video counts to the date ranges table
df_weeks['videos_count'] = videos_count_list

In [ ]:
df_weeks.to_csv("date_ranges_ATV_4.csv")
df4.to_csv("ATV_weeks.csv")

## Half-week periods (Not needed, no weeks with more than 50)

In [ ]:
df_weeks

In [ ]:
#splitting to half-weekly periods
split_periods = []

df_filtered = df_weeks[df_weeks["videos_count"] == 50].copy()

# Convert 'start' and 'end' to datetime
df_filtered["start"] = pd.to_datetime(df_filtered["start"])
df_filtered["end"] = pd.to_datetime(df_filtered["end"])

# Create new DataFrame for half-month periods
split_periods = []

for _, row in df_filtered.iterrows():
    start_date = row["start"]
    mid_date = start_date + timedelta(days=4)  # Mid-month
    end_date = row["end"]

    # Ensure mid_date does not exceed end_date
    if mid_date > end_date:
        mid_date = end_date

    # First half
    split_periods.append({"start": start_date, "end": mid_date - timedelta(seconds=1)})

    # Second half
    split_periods.append({"start": mid_date, "end": end_date})

# Create new DataFrame with split periods
df_half_weeks = pd.DataFrame(split_periods)

df_half_weeks["start"] = df_half_weeks["start"].dt.strftime("%Y-%m-%dT%H:%M:%S-05:00")
df_half_weeks["end"] = df_half_weeks["end"].dt.strftime("%Y-%m-%dT%H:%M:%S-05:00")

print(df_half_weeks)

In [ ]:
print(df_filtered.head())

In [ ]:
count50 = df_weeks[df_weeks["videos_count"] == 50]
count50.shape

In [ ]:
# Getting the youtube videos for the half weekly periods
channel_id = "UCpSJ5fGhmAME9Kx2D3ZvN3Q" # Latina Noticias
max_results = 50
order = "date"
keyword = "robo|robó|roban|robaron|delincuentes|criminales|asalto|asaltó|asaltan|asaltaron|matan|mató|mataron|ladrón|asesinato|asesinado|asesinan|asesinaron|extorsionadores|extorsionador|extorsionan|extorsionaron|extorsionó|secuestro|secuestradores|secuestrada|secuestrado|secuestró|secuestraron|balacera|balean|balearon|balazos|sicariato|mafias|mafia|sicarios"

# Create empty dataframe for first time running it
df = pd.DataFrame(columns=['video_id','title','url','channel','keyword','video_date'])
videos_list = []
videos_count_list = []

for index, row in df_half_weeks.iterrows():
    published_after = row['start']
    published_before = row['end']


    all = get_youtube_videos(api_key, channel_id, keyword, max_results,
                                                          order, published_after, published_before)
    videos = all[0]
    video_count = all[1]
    videos_list.append(videos)
    videos_count_list.append(video_count)

In [ ]:
#creating a df with the outputs
flattened_list = np.concatenate(videos_list).tolist()
df5 = pd.DataFrame(flattened_list)
df5.drop(columns=['keyword'], inplace=True)

In [ ]:
#adding the video counts to the date ranges table
df_half_weeks['videos_count'] = videos_count_list

In [ ]:
df_half_weeks.to_csv("date_ranges_4.csv")
files.download('date_ranges_4.csv')
#df5.to_csv("video_titles_03.09.3.csv")
#files.download('video_titles_03.09.3.csv')

## Couple days periods (not needed)

In [ ]:
uploaded = files.upload()

In [ ]:
df_half_weeks = pd.read_csv("date_ranges_4.csv")

In [ ]:
#splitting to half-weekly periods
split_periods = []

df_filtered = df_half_weeks[df_half_weeks["videos_count"] == 50].copy()

# Convert 'start' and 'end' to datetime
df_filtered["start"] = pd.to_datetime(df_filtered["start"])
df_filtered["end"] = pd.to_datetime(df_filtered["end"])

# Create new DataFrame for half-month periods
split_periods = []

for _, row in df_filtered.iterrows():
    start_date = row["start"]
    mid_date = start_date + timedelta(days=2)
    end_date = row["end"]

    # Ensure mid_date does not exceed end_date
    if mid_date > end_date:
        mid_date = end_date

    # First half
    split_periods.append({"start": start_date, "end": mid_date - timedelta(seconds=1)})

    # Second half
    split_periods.append({"start": mid_date, "end": end_date})

# Create new DataFrame with split periods
df_couple_days = pd.DataFrame(split_periods)

df_couple_days["start"] = df_couple_days["start"].dt.strftime("%Y-%m-%dT%H:%M:%S-05:00")
df_couple_days["end"] = df_couple_days["end"].dt.strftime("%Y-%m-%dT%H:%M:%S-05:00")

print(df_couple_days)

In [ ]:
#validating that the number of periods id correct
count50 = df_half_weeks[df_half_weeks["videos_count"] == 50]
count50.shape

In [ ]:
# Getting the youtube videos for the half weekly periods
channel_id = "UCpSJ5fGhmAME9Kx2D3ZvN3Q" # Latina Noticias
max_results = 50
order = "date"
keyword = "robo|robó|roban|robaron|delincuentes|criminales|asalto|asaltó|asaltan|asaltaron|matan|mató|mataron|ladrón|asesinato|asesinado|asesinan|asesinaron|extorsionadores|extorsionador|extorsionan|extorsionaron|extorsionó|secuestro|secuestradores|secuestrada|secuestrado|secuestró|secuestraron|balacera|balean|balearon|balazos|sicariato|mafias|mafia|sicarios"

# Create empty dataframe for first time running it
df = pd.DataFrame(columns=['video_id','title','url','channel','keyword','video_date'])
videos_list = []
videos_count_list = []

for index, row in df_couple_days.iterrows():
    published_after = row['start']
    published_before = row['end']


    all = get_youtube_videos(api_key, channel_id, keyword, max_results,
                                                          order, published_after, published_before)
    videos = all[0]
    video_count = all[1]
    videos_list.append(videos)
    videos_count_list.append(video_count)

In [ ]:
#creating a df with the outputs
flattened_list = np.concatenate(videos_list).tolist()
df6 = pd.DataFrame(flattened_list)
df6.drop(columns=['keyword'], inplace=True)

In [ ]:
#adding the video counts to the date ranges table
df_couple_days['videos_count'] = videos_count_list

In [ ]:
df_couple_days.to_csv("date_ranges_5.csv")
files.download('date_ranges_5.csv')
df6.to_csv("video_titles_03.09.4.csv")
files.download('video_titles_03.09.4.csv')

## Combining all the video that have not already been collected

In [ ]:
#DFs to combine: ATV_df_1, ATV_halfmonths_2, atv_weeks

ATV_df_1 = pd.read_csv("ATV_df_1.csv")
ATV_df_1= ATV_df_1.drop(columns=['Unnamed: 0'])
ATV_df_1.head()

In [ ]:
# ATV_halfmonths_2 = pd.read_csv("ATV_halfmonths_2.csv")
ATV_halfmonths_2.head()

In [ ]:
# df4 = pd.read_csv("ATV_weeks.csv")
df4.head()

In [ ]:
print(df4.shape)
print(ATV_halfmonths_2.shape)
print(ATV_df_1.shape)

In [ ]:
combined_ATV_df = pd.concat([df4, ATV_halfmonths_2, ATV_df_1])
combined_ATV_df.shape

In [ ]:
#removing duplicates
combined_ATV_df = combined_ATV_df.drop_duplicates(subset="video_id", keep="first")
combined_ATV_df = combined_ATV_df.reset_index(drop=True)
combined_ATV_df

In [ ]:
combined_ATV_df.to_csv("combined_ATV_df.csv")

# Collecting videos from El Comercio

## Full month periods

In [ ]:
#Dataframe with full month periods

# Define the start and end dates
start_date = datetime(2019, 2, 1)
end_date = datetime(2025, 3, 1)

# Create a list to hold the data
data = []

# Generate the start and end date-times for each full month period
current_date = start_date
while current_date < end_date:
    # Get the start date of the current month
    start_date_str = current_date.strftime("%Y-%m-%dT%H:%M:%S-05:00")

    # Calculate the next month's first day for the end date
    next_month = current_date.replace(day=28) + timedelta(days=4)  # this gives us the next month
    end_date_str = next_month.replace(day=1) - timedelta(seconds=1)
    end_date_str = end_date_str.strftime("%Y-%m-%dT%H:%M:%S-05:00")

    # Append to the data list
    data.append([start_date_str, end_date_str])

    # Move to the first day of the next month
    current_date = next_month.replace(day=1)

date_ranges = pd.DataFrame(data, columns=["start", "end"])

date_ranges


In [ ]:
# applying new function
# Create a for loop that goes through each time frame
    # gets all youtube videos and store them in a dataframe

channel_id = "UCLtGUPjKLqa3zgdmhKCZONg" # El Comercio
max_results = 50
order = "date"
keyword = "robo|robó|roban|robaron|delincuentes|criminales|asalto|asaltó|asaltan|asaltaron|matan|mató|mataron|ladrón|asesinato|asesinado|asesinan|asesinaron|extorsionadores|extorsionador|extorsionan|extorsionaron|extorsionó|secuestro|secuestradores|secuestrada|secuestrado|secuestró|secuestraron|balacera|balean|balearon|balazos|sicariato|mafias|mafia|sicarios"

# Create empty dataframe for first time running it
df = pd.DataFrame(columns=['video_id','title','url','channel','keyword','video_date'])
videos_list = []
videos_count_list = []

for index, row in date_ranges.iterrows():
    published_after = row['start']
    published_before = row['end']


    all = get_youtube_videos(api_key, channel_id, keyword, max_results,
                                                          order, published_after, published_before)
    videos = all[0]
    video_count = all[1]
    videos_list.append(videos)
    videos_count_list.append(video_count)


In [ ]:
flattened_list = np.concatenate(videos_list).tolist()
df_R_1 = pd.DataFrame(flattened_list)
df_R_1.drop(columns=['keyword'], inplace=True)

date_ranges['videos_count'] = videos_count_list

In [ ]:
df_R_1.to_csv("df_EC_1.csv")
date_ranges.to_csv("date_ranges_EC_1.csv")

## Week periods (not needed, no month has over 50)

# Combining all videos

In [ ]:
# El Comercio
df_EC = pd.read_csv("df_EC_1.csv")
df_EC.head()

In [ ]:
# ATV
df_ATV = pd.read_csv("combined_ATV_df.csv")
df_ATV.head()

In [ ]:
print(df_ATV.shape)
print(df_EC.shape)

In [ ]:
#combining ATV and El Comercio
combined_df = pd.concat([df_ATV, df_EC])
combined_df= combined_df.drop(columns=['Unnamed: 0'])
combined_df

In [ ]:
#removing duplicates
combined_df = combined_df.drop_duplicates(subset="video_id", keep="first")
combined_df = combined_df.reset_index(drop=True)
combined_df

# Transripts

In [ ]:
# for test
video_id = # Video ID removed

In [ ]:
# trnascript function that stores also the error messages

def get_transcript(video_id, language='es'):
    try:
        transcript = YouTubeTranscriptApi.get_transcript(video_id, languages=[language])
        return " ".join([line['text'] for line in transcript])  # Join transcript into a single string
    except Exception as e:
        return f"Error: {e}"  # Return error message

In [ ]:
get_transcript(video_id)

In [ ]:
# getting transcripts for videos in full df
combined_df['transcripts'] = [
    get_transcript(video_id) if time.sleep(2) is None else None
    for video_id in tqdm(combined_df['video_id'], desc="Processing transcripts")
]

In [ ]:
# Save to csv as backup
combined_df.to_csv("ATV_EC_transcripts_1.csv")

In [ ]:
combined_df = pd.read_csv("ATV_EC_transcripts_1.csv")
combined_df.shape

In [ ]:
# Filter out rows where the transcript contains an error message
df_errors = combined_df[combined_df['transcripts'].str.contains("Error:", na=False)].copy()

In [ ]:
df_errors.shape

In [ ]:
# Check types of errors
df_test = df_errors.copy()

error_count = df_test[df_test['transcripts'].str.startswith("Error", na=False)].shape[0]
print(f"Total transcripts with errors: {error_count}")
error_count = df_test[df_test['transcripts'].str.contains('age-restricted', case=False, na=False)].shape[0]
print(f"Total transcripts with age-restricted errors: {error_count}")
error_count = df_test[df_test['transcripts'].str.contains('Subtitles are disabled', case=False, na=False)].shape[0]
print(f"Total transcripts with subtitles are disabled errors: {error_count}")
error_count = df_test[df_test['transcripts'].str.contains('video is unplayable', case=False, na=False)].shape[0]
print(f"Total transcripts with video is unplayable errors: {error_count}")
error_count = df_test[df_test['transcripts'].str.contains('requested language codes:', case=False, na=False)].shape[0]
print(f"Total transcripts with requested language codes: ['es'] errors: {error_count}")

In [ ]:
# Remove rows with errors

combined_df = combined_df[~combined_df['transcripts'].str.startswith("Error:")]
combined_df.shape

In [ ]:
# Download to csv
combined_df.to_csv("ATV_EC_transcripts_1.csv")